___ 
# data
___

In [3]:
import os
from datetime import datetime
import numpy as np
import cv2
from glob import glob
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [36]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [37]:
def load_data(path, split=0.1):
    
    images = sorted(glob(os.path.join(path, 'MP6843_img/*')))
    masks = glob(os.path.join(path, 'MP6843_mask/*'))
    masks = masks + masks
    masks.sort()

    total_size = len(images)
    valid_size = int(split * total_size)
    test_size = int(split * total_size)

    train_x, valid_x = train_test_split(images, test_size=valid_size, random_state=42)
    train_y, valid_y = train_test_split(masks, test_size=valid_size, random_state=42)

    train_x, test_x = train_test_split(train_x, test_size=test_size, random_state=42)
    train_y, test_y = train_test_split(train_y, test_size=test_size, random_state=42)

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)


In [38]:
def read_image(path):
    
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (0,0), fx=0.5, fy=0.5)
    image = image/255.0
    
    return image

In [39]:
def read_mask(path):
    
    path = path.decode()
    mask = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
    mask = mask/255.0
    mask = np.expand_dims(mask, axis=-1)
    
    return mask

In [40]:
def tf_parse(image, mask):
    
    def _parse(image, mask):
        image = read_image(image)
        mask = read_mask(mask)
        
        return image, mask

    image, mask = tf.numpy_function(_parse, [image, mask], [tf.float64, tf.float64])
    image.set_shape([520, 696, 3])
    mask.set_shape([520, 696, 1])
   
    return image, mask

In [41]:
def tf_dataset(image, mask, batch=4):
    
    dataset = tf.data.Dataset.from_tensor_slices((image, mask))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch)
    dataset = dataset.repeat()
    
    return dataset

In [42]:
path = "/content/drive/MyDrive/DIPLOMSKI/code/dataset/"
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_data(path)

In [43]:
#for image, mask in train_dataset.take(1):
#  print("Image shape: ", image.numpy().shape)
#  image = image.numpy()
#  image = np.reshape(image, (-1, 520, 696, 3))
#  print("Label: ", mask.numpy().shape)

In [44]:
logdir = '/content/drive/MyDrive/DIPLOMSKI/code/logs/'
!rm -rf '/content/drive/MyDrive/DIPLOMSKI/code/logs/'

In [45]:
#logdir += 'train_data/' + datetime.now().strftime("%Y%m%d-%H%M%S")
#file_writer = tf.summary.create_file_writer(logdir)
#
#with file_writer.as_default():
#  tf.summary.image("Training data", image, step=0)

In [46]:
# %tensorboard --logdir /content/drive/MyDrive/DIPLOMSKI/code/logs/train_data

___
# model
___

In [47]:
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model
from types import MethodType
import keras.backend as K

In [48]:
def build_model(n_classes=1, input_height=520, input_width=696, 
                kernel=3, dropout=0.3, pool_size = 2):

    ## Encoder
    filter_size = 64
    img_input = Input(shape=(input_height, input_width, 3))

    x = img_input
    levels = []
    
    x = Conv2D(filter_size, (kernel, kernel), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout)(x)
    x = Conv2D(filter_size, (kernel, kernel), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    levels.append(x)
    
    x = MaxPooling2D((pool_size, pool_size))(x)
    
    x = Conv2D(128, (kernel, kernel), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout)(x)
    x = Conv2D(128, (kernel, kernel), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    levels.append(x)
    
    x = MaxPooling2D((pool_size, pool_size))(x)
    
    x = Conv2D(256, (kernel, kernel), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout)(x)
    x = Conv2D(256, (kernel, kernel), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    levels.append(x)
    
    x = MaxPooling2D((pool_size, pool_size))(x)
   
    x = Conv2D(512, (kernel, kernel), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout)(x)
    x = Conv2D(512, (kernel, kernel), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = ZeroPadding2D(((0, 1), (0, 1)))(x)
    levels.append(x)
    
    x = MaxPooling2D((pool_size, pool_size))(x)
    
    x = Conv2D(1024, (kernel, kernel), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = Dropout(dropout)(x)
    x = Conv2D(1024, (kernel, kernel), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    levels.append(x)
    
    [f1, f2, f3, f4, f5] = levels
    #for _ in levels:
      #print(_)

    ### Decoder
    o = f5
    
    o = Conv2DTranspose(512, kernel_size=(2, 2), strides=(2,2))(o)
    
    o = concatenate([o, f4], axis=-1)
    o = Conv2D(512, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)
    o = Dropout(dropout)(o)
    o = Conv2D(512, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)
    o = Cropping2D(((0, 1), (0, 1)))(o)
    
    o = Conv2DTranspose(256, kernel_size=(2, 2), strides=(2,2))(o)
        
    o = concatenate([o, f3], axis=-1)
    o = Conv2D(256, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)
    o = Dropout(dropout)(o)
    o = Conv2D(256, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)

    o = Conv2DTranspose(128, kernel_size=(2, 2), strides=(2,2))(o)

    o = concatenate([o, f2], axis=-1)
    o = Conv2D(128, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)
    o = Dropout(dropout)(o)
    o = Conv2D(128, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)

    o = Conv2DTranspose(64, kernel_size=(2, 2), strides=(2,2))(o)

    o = concatenate([o, f1], axis=-1)
    o = Conv2D(64, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)
    o = Dropout(dropout)(o)
    o = Conv2D(64, (3, 3), padding='same')(o)
    o = BatchNormalization()(o)
    o = Activation('relu')(o)

    ### Output
    o = Conv2D(n_classes, (1, 1), padding='same')(o)
    o = Activation('sigmoid')(o)
    model = Model(img_input, o)
    
    return model
    

In [49]:
model = build_model()
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 520, 696, 3) 0                                            
__________________________________________________________________________________________________
conv2d_57 (Conv2D)              (None, 520, 696, 64) 1792        input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_54 (BatchNo (None, 520, 696, 64) 256         conv2d_57[0][0]                  
__________________________________________________________________________________________________
activation_57 (Activation)      (None, 520, 696, 64) 0           batch_normalization_54[0][0]     
____________________________________________________________________________________________

In [50]:
from tensorflow.keras.utils import plot_model

In [51]:
#dot_img_file = '/content/drive/MyDrive/DIPLOMSKI/code/logs/model_0.png'
#tf.keras.utils.plot_model(model, to_file=dot_img_file, show_shapes=True)

---
# train
---



In [52]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger, TensorBoard

In [53]:
def iou(y_true, y_pred):
     
     def f(y_true, y_pred):
         
         intersection = (y_true * y_pred).sum()
         union = y_true.sum() + y_pred.sum() - intersection
         x = (intersection + 1e-15) / (union + 1e-15)
         x = x.astype(np.float32)
         
         return x
     
     return tf.numpy_function(f, [y_true, y_pred], tf.float32)

In [54]:
lr = 1e-4
epochs = 20
batch = 4

In [55]:
train_dataset = tf_dataset(train_x, train_y)
valid_dataset = tf_dataset(valid_x, valid_y)

In [56]:
model = build_model()

opt = tf.keras.optimizers.Adam(lr)
metrics = ["acc", tf.keras.metrics.Recall(), tf.keras.metrics.Precision(), iou]
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=metrics)

In [57]:
callbacks = [
          ModelCheckpoint("/content/drive/MyDrive/DIPLOMSKI/code/files/model_new.h5"),
          ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
          CSVLogger("/content/drive/MyDrive/DIPLOMSKI/code/files/data.csv"),
          TensorBoard(),
          EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=False)
     ]

In [58]:
train_steps = len(train_x)//batch
valid_steps = len(valid_x)//batch

if len(train_x) % batch != 0:
    train_steps += 1

if len(valid_x) % batch != 0:
    valid_steps += 1

model.fit(train_dataset,
    validation_data=valid_dataset,
    epochs=epochs,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks)

model.save("/content/drive/MyDrive/DIPLOMSKI/code/files/saved_model.h5")

Epoch 1/20
40/40 [==============================] - 352s 9s/step - loss: 0.5153 - acc: 0.8266 - recall_2: 0.3582 - precision_2: 0.6741 - iou: 0.2484 - val_loss: 0.5495 - val_acc: 0.7818 - val_recall_2: 0.0000e+00 - val_precision_2: 0.0000e+00 - val_iou: 0.1443
Epoch 2/20
40/40 [==============================] - 70s 2s/step - loss: 0.2688 - acc: 0.9064 - recall_2: 0.7028 - precision_2: 0.8442 - iou: 0.4749 - val_loss: 0.5438 - val_acc: 0.7818 - val_recall_2: 0.0000e+00 - val_precision_2: 0.0000e+00 - val_iou: 0.0922
Epoch 3/20
40/40 [==============================] - 71s 2s/step - loss: 0.2545 - acc: 0.9095 - recall_2: 0.7044 - precision_2: 0.8578 - iou: 0.4915 - val_loss: 0.6654 - val_acc: 0.6113 - val_recall_2: 0.4346 - val_precision_2: 0.2633 - val_iou: 0.1741
Epoch 4/20
40/40 [==============================] - 70s 2s/step - loss: 0.2455 - acc: 0.9111 - recall_2: 0.7104 - precision_2: 0.8601 - iou: 0.5032 - val_loss: 0.7317 - val_acc: 0.7818 - val_recall_2: 0.0000e+00 - val_precision

___
#predict
___

In [59]:
from tensorflow.keras.utils import CustomObjectScope
from tqdm import tqdm

In [60]:
test_dataset = tf_dataset(test_x, test_y)
 
test_steps = (len(test_x)//batch)
if len(test_x) % batch != 0:
  test_steps += 1

In [61]:
#reloaded_model = tf.keras.models.load_model('model.h5')

In [64]:
test_ds = tf_dataset(test_x, test_y)

In [67]:
test_steps = (len(test_x)//batch)
if len(test_x) % batch != 0:
  test_steps += 1

In [69]:
#model.load_weights('model.h5')

ValueError: ignored

In [90]:
def mask_parse(mask):
     mask = np.squeeze(mask)
     mask = [mask, mask]
     mask = np.transpose(mask, (1, 2, 0))
     return mask

def read_image(path):
     x = cv2.imread(path, cv2.IMREAD_COLOR)
     x = cv2.resize(x, (0,0), fx=0.5, fy=0.5)
     x = x/255.0
     return x

def read_mask(path):
     x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)
     #x = cv2.resize(x, (256, 256))
     x = np.expand_dims(x, axis=-1)
     return x

In [111]:
for i, (x, y) in tqdm(enumerate(zip(test_x, test_y)), total=len(test_x)):
  x = read_image(x)
  y = read_mask(y)
  y_pred = model.predict(np.expand_dims(x, axis=0))[0] > 0.5
  y_pred = y_pred.astype(np.uint8) * 255.0
  #print(y_pred)
  cv2.imwrite(f"/content/drive/MyDrive/DIPLOMSKI/code/results/{i}_GT.png", y)
  cv2.imwrite(f"/content/drive/MyDrive/DIPLOMSKI/code/results/{i}_PRED.png", y_pred)

100%|██████████| 20/20 [00:03<00:00,  5.13it/s]


In [2]:
tf

NameError: ignored